In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
conn = sqlite3.connect(r"c:\Dev\06. Data\3. FDA NCTs - SQL\studies_db_06-Nov-2018_all date fields as ints.sqlite3")
df = pd.read_sql_query("SELECT * FROM location_countries", conn)
df.head()

,id,nct_id,country
0,1,NCT00000102,United States of America
1,2,NCT00000104,United States of America
2,3,NCT00000105,United States of America
3,4,NCT00000106,United States of America
4,5,NCT00000107,United States of America


In [3]:
df_countries_list = pd.read_sql_query("SELECT DISTINCT country FROM location_countries ORDER BY country ASC", conn)
df_countries_list.to_csv(r"c:\Dev\06. Data\3. FDA NCTs - SQL\countries_from_sql.csv")
df_countries_list.head()

,country
0,**None**
1,Afghanistan
2,Albania
3,Algeria
4,American Samoa


In [16]:
country_phase_by_year = pd.read_sql_query("""
    SELECT Count(s1.nct_id) as nct_id, strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch')) as year
            FROM studies as s1
            WHERE s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
            GROUP BY strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch'))  
            ORDER BY year ASC
""", conn)
country_phase_by_year.head()

,nct_id,year
0,684,1999
1,255,2000
2,231,2001
3,181,2002
4,233,2003


In [19]:
country_phase_by_year.index = country_phase_by_year.year
country_phase_by_year.drop(columns=["year"], axis=1, inplace=True)
country_phase_by_year.head()

,nct_id
year,
1999,684
2000,255
2001,231
2002,181
2003,233


In [22]:
print(country_phase_by_year.to_dict())

{'nct_id': {'1999': 684, '2000': 255, '2001': 231, '2002': 181, '2003': 233, '2004': 247, '2005': 844, '2006': 1011, '2007': 1514, '2008': 2179, '2009': 2460, '2010': 2300, '2011': 2398, '2012': 2329, '2013': 2218, '2014': 3019, '2015': 2437, '2016': 2493, '2017': 2418, '2018': 2015}}


In [27]:
df_studies_phases_by_year = pd.read_sql_query("""
    SELECT Count(s.nct_id) as studiesNum, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as month, 
        ph2.phaseTwo, CAST(ph.phaseOne AS INT) as phaseOne, ph3.phaseThree, ph4.phaseFour, ph_unk.phaseUnk
    FROM studies as s
        LEFT JOIN 
        (
            SELECT Count(s1.nct_id) as phaseOne, strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch')) as s1month
            FROM studies as s1
            WHERE s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
            GROUP BY strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch'))    
        ) as ph
        ON ph.s1month = strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
        LEFT JOIN 
        (
            SELECT Count(s2.nct_id) as phaseTwo, strftime('%Y', datetime(s2.study_first_submitted, 'unixepoch')) as s1month
            FROM studies as s2
            WHERE s2.phase = 'Phase 2' OR s2.phase = 'Phase 1/Phase 2'
            GROUP BY strftime('%Y', datetime(s2.study_first_submitted, 'unixepoch'))    
        ) as ph2
        ON ph2.s1month = strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
                
        LEFT JOIN 
        (
            SELECT Count(s3.nct_id) as phaseThree, strftime('%Y', datetime(s3.study_first_submitted, 'unixepoch')) as s1month
            FROM studies as s3
            WHERE s3.phase = 'Phase 3' or s3.phase = 'Phase 2/Phase 3'
            GROUP BY strftime('%Y', datetime(s3.study_first_submitted, 'unixepoch'))    
        ) as ph3
        ON ph3.s1month = strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
        
        LEFT JOIN 
        (
            SELECT Count(s4.nct_id) as phaseFour, strftime('%Y', datetime(s4.study_first_submitted, 'unixepoch')) as s1month
            FROM studies as s4
            WHERE s4.phase = 'Phase 4'
            GROUP BY strftime('%Y', datetime(s4.study_first_submitted, 'unixepoch'))    
        ) as ph4
        ON ph4.s1month = strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
        
        LEFT JOIN 
        (
            SELECT Count(s5.nct_id) as phaseUnk, strftime('%Y', datetime(s5.study_first_submitted, 'unixepoch')) as s1month
            FROM studies as s5
            WHERE s5.phase = '**None**' or s5.phase = 'N/A'
            GROUP BY strftime('%Y', datetime(s5.study_first_submitted, 'unixepoch'))    
        ) as ph_unk
        ON ph_unk.s1month = strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
        
    WHERE strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) < '2018-06'    
    GROUP BY strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
    """, conn)
df_studies_phases_by_year.head()

,studiesNum,month,phaseTwo,phaseOne,phaseThree,phaseFour,phaseUnk
0,3645,1999,1271,684,740,54,896
1,1980,2000,571,255,236,28,890
2,1346,2001,531,231,174,29,381
3,1588,2002,652,181,387,69,299
4,1662,2003,677,233,333,50,369


In [31]:
df_studies_phases_by_year.index = df_studies_phases_by_year.month
print(df_studies_phases_by_year.to_dict(orient="index"))

{'1999': {'studiesNum': 3645, 'month': '1999', 'phaseTwo': 1271, 'phaseOne': 684, 'phaseThree': 740, 'phaseFour': 54, 'phaseUnk': 896}, '2000': {'studiesNum': 1980, 'month': '2000', 'phaseTwo': 571, 'phaseOne': 255, 'phaseThree': 236, 'phaseFour': 28, 'phaseUnk': 890}, '2001': {'studiesNum': 1346, 'month': '2001', 'phaseTwo': 531, 'phaseOne': 231, 'phaseThree': 174, 'phaseFour': 29, 'phaseUnk': 381}, '2002': {'studiesNum': 1588, 'month': '2002', 'phaseTwo': 652, 'phaseOne': 181, 'phaseThree': 387, 'phaseFour': 69, 'phaseUnk': 299}, '2003': {'studiesNum': 1662, 'month': '2003', 'phaseTwo': 677, 'phaseOne': 233, 'phaseThree': 333, 'phaseFour': 50, 'phaseUnk': 369}, '2004': {'studiesNum': 1787, 'month': '2004', 'phaseTwo': 708, 'phaseOne': 247, 'phaseThree': 482, 'phaseFour': 61, 'phaseUnk': 289}, '2005': {'studiesNum': 12895, 'month': '2005', 'phaseTwo': 3252, 'phaseOne': 844, 'phaseThree': 3248, 'phaseFour': 1766, 'phaseUnk': 3785}, '2006': {'studiesNum': 10926, 'month': '2006', 'phaseT

In [26]:
PHASES_DICT = {
    "unknown" : ["**None**", "N/A"],
    "phase1": ["Phase 1", "Early Phase 1"],
    "phase2": ["Phase 2", "Phase 1/Phase 2"],
    "phase3": ["Phase 3", "Phase 2/Phase 3"],
    "phase4": ["Phase 4"]
}
cond_preamb = "s1.phase = '"
cond_concat = " or "
for country in df_countries_list.country:
    print("Country:{}".format(country))
    for phase in PHASES_DICT:
        where_condition = ""
        phase_condition_array = PHASES_DICT[phase]
        i=0
        while True:
            where_condition += cond_preamb+phase_condition_array[i]+"'"
            i+=1
            if i<len(phase_condition_array):
                where_condition += cond_concat
                continue
            else: break
        print("Phase:{}    Condition:{}".format(phase, where_condition))
        
    

Country:**None**
Phase:unknown    Condition:s1.phase = '**None**' or s1.phase = 'N/A'
Phase:phase1    Condition:s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
Phase:phase2    Condition:s1.phase = 'Phase 2' or s1.phase = 'Phase 1/Phase 2'
Phase:phase3    Condition:s1.phase = 'Phase 3' or s1.phase = 'Phase 2/Phase 3'
Phase:phase4    Condition:s1.phase = 'Phase 4'
Country:Afghanistan
Phase:unknown    Condition:s1.phase = '**None**' or s1.phase = 'N/A'
Phase:phase1    Condition:s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
Phase:phase2    Condition:s1.phase = 'Phase 2' or s1.phase = 'Phase 1/Phase 2'
Phase:phase3    Condition:s1.phase = 'Phase 3' or s1.phase = 'Phase 2/Phase 3'
Phase:phase4    Condition:s1.phase = 'Phase 4'
Country:Albania
Phase:unknown    Condition:s1.phase = '**None**' or s1.phase = 'N/A'
Phase:phase1    Condition:s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
Phase:phase2    Condition:s1.phase = 'Phase 2' or s1.phase = 'Phase 1/Phase 2'
Phase:phase3    Cond